# <span style="color:#336699">SER-347 - Introdução à Programação para Sensoriamento Remoto</span>
<hr style="border:2px solid #0077b9;">

# <span style="color:#336699">Leitura e Escrita de Dados Vetoriais - Parte I: Usando a Biblioteca Fiona</span>


[<img src="img/fiona/feature-collection.png" alt="Coleção de Feições" width="150" align="right">](https://www.dpi.inpe.br)


- Gilberto Ribeiro de Queiroz
- Thales Sehn Körting
- Fabiano Morelli

In [ ]:
!ls -l data

In [ ]:
import fiona

fiona.__version__

In [ ]:
from shapely.geometry import shape
from shapely.geometry import mapping

In [ ]:
with fiona.open("data/deter/deter_201707.shp", "r") as deter:

    # Número de feições
    num_features = len( deter )

    print( "Número de feições: {}\n". format( num_features ) )


    # Sistema de Referência Espacial
    crs = deter.crs

    print( "CRS: {}\n".format(crs) )


    # Extensão da coleção de feições
    mbr = deter.bounds

    print( "xmin: {}, xmax: {}".format(mbr[0], mbr[2]) )
    print( "ymin: {}, ymax: {}\n".format(mbr[1], mbr[3]) )


    # Esquema das feições
    for k, v in deter.schema["properties"].items():
        print( "Atributo: {}, Tipo: {}".format(k, v) )

    print("\nTipo do atributo geométrico: {}\n". format(deter.schema["geometry"]) )


    # Acessando cada uma das feições
    for feature in deter:
        # obtendo a geometria associada a feição
        geom = shape( feature["geometry"] )

        # obtendo o atributo de data associado a feição
        view_date = feature["properties"]["view_date"]

        # computando o centróide da geometria recuperada
        centroide = geom.centroid

        print( "view_date: {}, Localização: {}".format(view_date, centroide.wkt) )

In [ ]:
with fiona.open('data/uf-2018/BRUFE250GC_SIR.shp', 'r') as uf:
    source_driver = uf.driver
    source_crs = uf.crs
    source_schema = uf.schema

In [ ]:
source_driver

In [ ]:
source_crs

In [ ]:
source_schema

In [ ]:
type(source_schema)

In [ ]:
new_schema = dict(source_schema)

In [ ]:
new_schema["geometry"] = "Point"

In [ ]:
new_schema

In [ ]:
with fiona.open("data/centroide.shp", "w",
                driver=source_driver,
                crs=source_crs,
                schema=new_schema) as centroides:
    with fiona.open("data/uf-2018/BRUFE250GC_SIR.shp", "r") as uf:

        for feature in uf:
            geom = shape( feature["geometry"] )

            centroide = geom.centroid

            feature["geometry"] = mapping(centroide)

            centroides.write(feature)